## Extraindo séries temporais para áreas de interesse

#### Este exemplo mostra como extrair séries de evapotranspiração média em pivôs centrais selecionados

### Importando a biblioteca Google Earth Engine

In [1]:
try:
    import ee 
except:
    !pip install earthengine-api
    import ee

In [2]:
# Instalando bibliotecas secundárias do OpenET

# Installing openet-core package
# https://github.com/Open-ET/openet-core

#    !pip install openet-core

# Installing openet.refetgee package
# https://github.com/Open-ET/openet-refet-gee

#    !pip install openet-refet-gee

### Inicializando Google Earth Engine
#### Necessário definir o projeto_id a ser utilizado

In [2]:
project_id = 'et-brasil'

try:
    ee.Initialize(project=project_id)
except:
    ee.Authenticate()

### Importando o modelo geeSEBAL e definido a localização do ambiente de trabalho 

In [3]:
import geopandas as gpd
import geemap
from tqdm import tqdm

import pprint
import sys
import datetime
import pandas as pd
import numpy as np
from IPython.display import Image

sys.path.append(r'C:/openet_brazil/workshop_et_models/openet-ptjpl')

import openet.ptjpl as model

### Variáveis

In [4]:
# Set landsat possible collections
collections = ['LANDSAT/LT04/C02/T1_L2',
               'LANDSAT/LT05/C02/T1_L2',
               'LANDSAT/LE07/C02/T1_L2',
               'LANDSAT/LC08/C02/T1_L2',
               'LANDSAT/LC09/C02/T1_L2']

# Set ET reference dataset and parameters
meteorology_source_inst = "ECMWF/ERA5_LAND/HOURLY"
meteorology_source_daily = "projects/openet/assets/meteorology/era5land/sa/daily"
et_reference_source = 'projects/et-brasil/assets/era5/daily/etr'
et_reference_band = 'eto24h'
et_reference_factor = 1
et_reference_resample = 'nearest'
et_reference_date_type = 'daily'

# Ponto de interesse
point = ee.Geometry.Point([-47.2425,-17.5372])

# Período de análise
start_date = '2020-01-01'
end_date = '2022-01-01'

# Cobertura de nuvens
cloud_cover = 50

# Lê arquivo shapefile das áreas de interesse
aoi_file = 'geometria/pivos_bacia_sao_marcos.shp'
aoi_info = gpd.read_file(aoi_file)
ee_aoi = geemap.geopandas_to_ee(aoi_info)

# Tamanho da imagem para a visualização
image_size = 768

### Criando o objeto do modelo

In [5]:
# Gera a Coleção de Imagens geeSEBAL
model_obj = model.Collection(
collections=collections,
start_date=start_date,
end_date=end_date,
geometry=ee_aoi.geometry(),
cloud_cover_max=cloud_cover,
model_args={
            'meteorology_source_inst': meteorology_source_inst,
            'meteorology_source_daily': meteorology_source_daily,
            'et_reference_source': et_reference_source,
            'et_reference_band': et_reference_band,
            'et_reference_factor': et_reference_factor,
            'et_reference_resample': et_reference_resample,
            'cold_calibration_points': 1,
            'hot_calibration_points': 1},)    

# Calcula variáveis para cada imagem da coleção
overpass_coll = model_obj.overpass(variables=['ndvi', 'lst', 'et'])

### Opção 1 multiplas tabelas cada uma contendo a série temporal de um pivô selecionado

In [6]:
# Loop between flux tower sites
for i in tqdm(aoi_info.index):

    # Seleciona pivô e guarda informações
    aoi = aoi_info.loc[i:i, :]
    id = aoi['PIVO_CODE'].values[0] # TODO: Atualizar o valor "PIVO_CODE" de acordo com o .shp da área de interesse

    # Converte GeoDataFrame em ee.Feature
    ee_aoi = geemap.geopandas_to_ee(aoi_info.loc[i:i,:]).first()

    # Função para extrair valores das bandas para um pivô
    def extract_band_values(image):
        try:
            values = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=ee_aoi.geometry(),
                scale=30
            )
        except:
            values = {'ndvi': np.nan, 'lst': np.nan, 'et': np.nan}
        
        return ee.Feature(None, values).set('date', image.date().format())

    # Aplicar a função em cada imagem da coleção
    time_series = overpass_coll.map(extract_band_values)

    # Exportar a série temporal como uma tabela para o Google Drive
    task = ee.batch.Export.table.toDrive(
        collection=time_series,
        folder= f'SerieTemporal_Pivos_ptjpl',
        description=f'ptjpl_pivo_{id}',
        selectors=['date', 'ndvi', 'lst', 'et'],
        fileFormat='CSV'
    )

    task.start()

100%|██████████| 88/88 [06:52<00:00,  4.69s/it]


### Opção 2 para exportar uma tabela com os resultados para um ponto (Utilizando Earth Engine)
#### Para períodos mais longos e modelos mais complexos (geesebal, eemetric)


In [ ]:
# Função para calcular ET média em cada pivô central
def extract_et(img):
    date_str = img.date().format('YYYY-MM-dd')
    
    # Aplica a média sobre cada feature
    stats = img.reduceRegions(
        collection=ee_aoi,
        reducer=ee.Reducer.mean(),
        scale=30  
    ).map(lambda f: f.set('date', date_str))
    
    return stats

# Extrai médias dos pivôs selecionados para cada imagem da coleção
results = overpass_coll.map(extract_et).flatten()

# Exportar a série temporal como uma tabela para o Google Drive
task = ee.batch.Export.table.toDrive(
    collection=results,
    folder= f'SerieTemporal_Pivos_ptjpl',
    description=f'ptjpl_agregado_pivos',
    fileFormat='CSV'
)
task.start()
